In [1]:
#Loading Original Naive RAG from previous work
"""
Parameter Experimentation
Testing different embedding models and retrieval strategies
"""
import sys
sys.path.insert(0, '../src')
from naive_rag import NaiveRAG
from utils import calculate_f1_score, exact_match
import pandas as pd
import time
from tqdm import tqdm

print("STEP 4: PARAMETER EXPERIMENTS")
print("\nBaseline: Naive RAG from previous implementation")
print("- Embedding: all-MiniLM-L6-v2 (384d)")
print("- Retrieval: top-1")
print("- Results: F1=0.444, EM=39.0%")
print("\nTesting variations in embedding dimensions and top_k retrieval and evaluating performance for \\\
comparision to NaiveRAG implementation\n")

# Prepare test set (50 questions for efficiency)
test_size = 50

def run_evaluation(rag_system, test_qs, top_k, label):
	"""Run evaluation on a RAG configuration"""
	results = []
	start = time.time()
	
	for qa in tqdm(test_qs, desc=label):
		try:
			result = rag_system.query(qa['question'], top_k=top_k, prompt_strategy="basic")
			f1 = calculate_f1_score(result['answer'], qa['answer'])
			em = exact_match(result['answer'], qa['answer'])
			results.append({'f1': f1, 'em': em})
		except:
			results.append({'f1': 0.0, 'em': False})
	
	elapsed = time.time() - start
	avg_f1 = sum(r['f1'] for r in results) / len(results)
	avg_em = sum(r['em'] for r in results) / len(results)
	
	return {
		'avg_f1': avg_f1,
		'avg_em': avg_em * 100,
		'time_min': elapsed
	}

Utils module loaded successfully!
STEP 4: PARAMETER EXPERIMENTS

Baseline: Naive RAG from previous implementation
- Embedding: all-MiniLM-L6-v2 (384d)
- Retrieval: top-1
- Results: F1=0.444, EM=39.0%

Testing variations in embedding dimensions and top_k retrieval and evaluating performance for \comparision to NaiveRAG implementation



In [3]:
#Experiment Matrix
embedding_models = [
	"all-MiniLM-L6-v2",   # Baseline (384d)
	"all-mpnet-base-v2"    # Higher quality (768d)
]

top_k_values = [1, 3, 5]

results_matrix = []

# Load test questions once
rag_temp = NaiveRAG()
_, qa_pairs = rag_temp.load_dataset()
test_questions = [qa_pairs[i] for i in range(test_size)]

print(f"Testing {len(embedding_models)} embedding models × {len(top_k_values)} retrieval strategies")
print(f"= {len(embedding_models)* len(top_k_values)} configurations")
print(f"The evaluations are run for each on {test_size} questions\n")

2025-10-02 03:36:17,068 - naive_rag - INFO - Initializing RAG with embedding model: all-MiniLM-L6-v2
2025-10-02 03:36:17,073 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: cpu
2025-10-02 03:36:17,074 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2025-10-02 03:36:18,314 - naive_rag - INFO - RAG system initialized successfully
2025-10-02 03:36:18,316 - naive_rag - INFO - Loading RAG Mini Wikipedia dataset...
2025-10-02 03:36:18,316 - naive_rag - INFO - Loading text corpus...
2025-10-02 03:36:20,480 - naive_rag - INFO - Loading Q&A pairs...
2025-10-02 03:36:21,913 - naive_rag - INFO - Loaded 3200 documents and 918 Q&A pairs


Testing 2 embedding models × 3 retrieval strategies
= 6 configurations
The evaluations are run for each on 50 questions



In [5]:
#Running Experiments
for emb_model in embedding_models:
	print("\n")
	print(f"Embedding Model: {emb_model}")
	print("\n")
	
	# Initialize RAG with this embedding model
	rag = NaiveRAG(embedding_model_name=emb_model)
	rag.load_dataset()
	rag.create_embeddings(batch_size=32)
	rag.build_vector_db()
	rag.load_generator()
	
	for top_k in top_k_values:
		label = f"{emb_model[:20]}... | top-{top_k}"
		result = run_evaluation(rag, test_questions, top_k, label)
		
		results_matrix.append({
			'Embedding Model': emb_model,
			'Top-K': top_k,
			'Avg F1': f"{result['avg_f1']:.3f}",
			'EM %': f"{result['avg_em']:.1f}%",
			'Time (min)': f"{result['time_min']:.1f}"
		})
		
		print(f"\nResults for top-{top_k}:")
		print(f"  F1: {result['avg_f1']:.3f}")
		print(f"  EM: {result['avg_em']:.1f}%")

2025-10-02 03:37:43,937 - naive_rag - INFO - Initializing RAG with embedding model: all-MiniLM-L6-v2
2025-10-02 03:37:43,941 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: cpu
2025-10-02 03:37:43,941 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2




Embedding Model: all-MiniLM-L6-v2




2025-10-02 03:37:44,922 - naive_rag - INFO - RAG system initialized successfully
2025-10-02 03:37:44,922 - naive_rag - INFO - Loading RAG Mini Wikipedia dataset...
2025-10-02 03:37:44,922 - naive_rag - INFO - Loading text corpus...
2025-10-02 03:37:46,723 - naive_rag - INFO - Loading Q&A pairs...
2025-10-02 03:37:47,998 - naive_rag - INFO - Loaded 3200 documents and 918 Q&A pairs
2025-10-02 03:37:47,998 - naive_rag - INFO - Creating embeddings for 3200 documents...


Batches:   0%|          | 0/100 [00:00<?, ?it/s]

2025-10-02 03:38:54,189 - naive_rag - INFO - Created embeddings with shape: (3200, 384)
2025-10-02 03:38:54,189 - naive_rag - INFO - Building FAISS vector database...
2025-10-02 03:38:54,189 - naive_rag - INFO - FAISS index built with 3200 vectors
2025-10-02 03:38:54,201 - naive_rag - INFO - Loading generation model: google/flan-t5-base
2025-10-02 03:38:54,831 - naive_rag - INFO - Generation model loaded successfully
all-MiniLM-L6-v2... | top-1:   0%|                                                              | 0/50 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

all-mpnet-base-v2... | top-5:  98%|██████████████████████████████████████████████████▉ | 49/50 [00:42<00:00,  1.31it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

all-mpnet-base-v2... | top-5: 100%|████████████████████████████████████████████████████| 50/50 [00:43<00:00,  1.16it/s]


Results for top-5:
  F1: 0.632
  EM: 56.0%


In [7]:
#Displaying and saving results
df = pd.DataFrame(results_matrix)

print("\n")
print("STEP 4: PARAMETER EXPERIMENT RESULTS\n")
print(df.to_string(index=False))

# Save results
df.to_csv('../results/04_parameter_experiments.csv', index=False)
print("\nResults saved to ../results/step4_parameter_experiments.csv")



STEP 4: PARAMETER EXPERIMENT RESULTS

  Embedding Model  Top-K Avg F1  EM % Time (min)
 all-MiniLM-L6-v2      1  0.516 46.0%        0.6
 all-MiniLM-L6-v2      3  0.516 46.0%        0.6
 all-MiniLM-L6-v2      5  0.516 46.0%        0.6
all-mpnet-base-v2      1  0.632 56.0%        0.7
all-mpnet-base-v2      3  0.632 56.0%        0.7
all-mpnet-base-v2      5  0.632 56.0%        0.7

Results saved to ../results/step4_parameter_experiments.csv
